This is a notebook version of guide that is shown in Azure AI Service / Agents

https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?source=recommendations&tabs=python&pivots=overview

in order to get the notebook to run correctly, please follow the steps to include bing as api connection.

you will also require to "login" as part of the token validation to the Azure AI Foundry project.

In [ ]:
! pip install -r requirements.txt

In [ ]:
!az logout
!az login

In [8]:
import os
import json

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool, MessageTextContent

from dotenv import load_dotenv

load_dotenv(override=True)

True

Define project client

In [9]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

Define Bing API Grounding Tool

In [11]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)
# print(bing.definitions)



/subscriptions/236600c3-67f1-473c-bc20-f50a678af342/resourceGroups/rg-jhl-aistudio/providers/Microsoft.MachineLearningServices/workspaces/jhl-agent-project/connections/bing-api


Running the Agent

In [12]:
# gpt model deployment name that is available in the foundry project
model_deployment_name = "gpt-4o-jhl-aiproj-agent"

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=model_deployment_name,
        name="my-assistant",
        instructions="You are a helpful assistant, Always use the Bing tool to ground the information.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_bing_grounding_tool]

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are some of the top news today?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Retrieve run step details to get Bing Search query link
    # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    # print(f"Last run step detail: {run_steps_data}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    # print(f"Messages: {messages}")

    # The messages are following in the reverse order,
    # we will iterate them and output only text contents.
    for data_point in reversed(messages.data):
        last_message_content = data_point.content[-1]
        if isinstance(last_message_content, MessageTextContent):
            print(f"{data_point.role}: {last_message_content.text.value}")

Created agent, ID: asst_8YOSyVp5EY33aFGRrDWdPyUd
Created thread, ID: thread_eqVBukOr0XaZ5ac15UcPvcP3
Created message, ID: msg_0q3dqS7K8GvonsLoJbrLwr2L
Run finished with status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: What are some of the top news today?
MessageRole.AGENT: Here are some of the top news stories for today, January 10, 2025:

1. **California Wildfires**: Multiple wildfires continue to rage across California, with significant damage and several fatalities reported【5†source】【9†source】.
2. **Donald Trump's Legal Situation**: President-elect Donald Trump received an unconditional discharge in his New York "hush money" trial conviction【5†source】.
3. **Meta Ends U.S. Fact-Checking Program**: Meta has announced the termination of its third-party fact-checking initiative for Facebook, Instagram, and Threads, replacing it with a “community notes” model【5†source】.
4. **New WSU Football Coach**: Washington State University has introduced a new head football coach【5†source